# Scrapper para obtener información de búsqueda de google (LinkdIn y/o Facebook)

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd

## Conexión al navegador

In [268]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://google.com'
driver.get(start_url)
# Modificar el tamaño de la ventana
#driver.execute_script("document.body.style.zoom='50%'")

## Lectura de archivos csv para obtener nombres

In [266]:
df_4 = pd.read_csv('ejemplo.csv', keep_default_na=False) 
df_4['NombreCompleto'] = df_4.apply(lambda row: f"{row['Nombre']} {row['Paterno']} {row['Materno']}", axis=1)
df_4

,Nombre,Paterno,Materno,NombreCompleto
0,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
1,Luis,Cruz,Jaramillo,Luis Cruz Jaramillo
2,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
3,José Alberto,Montán,López,José Alberto Montán López
4,Andres Manuel,Lopez,Obrador,Andres Manuel Lopez Obrador
5,Jorge Luis,Lopez,,Jorge Luis Lopez
6,Juan de Dios,Olivares,Jimenez,Juan de Dios Olivares Jimenez
7,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
8,José Alberto,Montán,López,José Alberto Montán López


# Scrapper

## Concatenado de etiquetas

In [ ]:
data_ = []
data = []

for i in df_4['NombreCompleto']:
    nombres.append(i)
    nombre = driver.find_element(By.XPATH, '//*[@id="APjFqb"]')
    nombre.send_keys(Keys.CONTROL + "a")
    nombre.send_keys(Keys.DELETE)
    nombre.send_keys(i)
    nombre.send_keys(Keys.ENTER)

    # Esperar a que los resultados de búsqueda se carguen completamente
    time.sleep(2)

    # Obtener el HTML de la página actual
    html_link = driver.page_source
    soup = BeautifulSoup(html_link, 'html.parser')

    # Extracción de clases
    div = soup.find_all('div', class_="kvH3mc BToiNc UK95Uc")
    cont = soup.find_all('div', class_="Z26q7c UK95Uc")

    # Primer bucle para extraer la clase que conforma el bloque que corresponde al URL desplegado
    if div:
        for tag in div:
            href_tags = tag.find_all('a', href=True)
            text = tag.text

            for href_tag in href_tags:
                href = href_tag['href']

                data.append({'href': href, 'Text': text, 'Nombre': i})
    else:
        data.append({'href': None, 'Text': None, 'Nombre': i})

    # Segundo bucle para extraer los resúmenes de las URL desplegadas
    if cont:
        for tag in cont:
            text = ''

            span_tags = tag.find_all('span')
            if span_tags:
                for span_tag in span_tags:
                    if span_tag.get('class', []) == ['ITHCWe']:
                        continue
                    text += span_tag.text.strip() + ' '
            else:
                text = tag.text.strip()

            data_.append({'Text': text.strip()})
    else:
        data_.append({'Text': None})

    google = driver.find_element(By.XPATH, '//*[@id="logo"]')
    google.click()

    # Rellenar los datos faltantes en data y data_ para tener la misma longitud
    max_length = max(len(data), len(data_))
    data += [{'href': None, 'Text': None}] * (max_length - len(data))
    data_ += [{'Text': None}] * (max_length - len(data_))

df = pd.DataFrame(data)
df_2 = pd.DataFrame(data_)

df_3 = pd.concat([df, df_2], axis=1)
df_3 = df_3[df_3['href'].str.contains('facebook|twitter|instagram|linkedin', case=False)]
df_3 = df_3[~df_3['href'].str.contains('translate.google')]


# Se guarda el df en un csv
df_3.to_csv('dataframe3.csv')
